# Analysis of Qualities of Anime
## Spring 2024 Data Science Project
Ryan Jiang and Chukwubuike Peterson


## Introduction
Japanese animation, or anime, has transcended boundaries and exploded in popularity in recent years. Once a niche interest in the West, a 2023 survey reveals a dedicated viewership: 25% of Millennials and a staggering 42% of Gen Zs watch anime weekly. This trend aligns with projections of the anime industry reaching a massive $62.7 billion by 2032. Anime's cultural impact extends beyond viewership - celebrities and artists openly reference it, further solidifying its place in popular culture. However, popularity paints an incomplete picture. As anime embeds itself within the next generation's cultural landscape, a critical data gap emerges. This has sparked an interest in answering the question: What makes anime appeal with viewers? 

Specifically recognizing that fan preferences constantly evolve, we want to know past trends and also predict what qualities resonate with viewers. Essentially, this study seeks to anticipate what stories and styles will capture audiences in this ever-evolving landscape of anime. Further analysis can reveal hidden connections between genres and popularity and can be leveraged to know which contents are more likely to be successful. Combining these findings from quantitative research with external qualitative research can offer a more complete understanding of anime's impact and its influence within society. Overall, this study will attempt to uncover any relationships between variables of anime and other factors. 
